In [1]:
import numpy as np
import pandas as pd
from tqdm import trange
import sklearn.metrics.pairwise as kernel
from copy import copy
import statsmodels.api as sm
import gc

In [3]:
data=pd.read_csv('F:/usdata/cha1.csv',index_col=0)

In [4]:
date=list(pd.unique(data.index))
date.sort()

date1=date[:400]
characteristics1=dict(zip(
    date1,
    [2*data.loc[i].set_index('PERMNO').iloc[:,1:].rank()/data.loc[i].shape[0]-1 for i in date1]
))
ret1=data.loc[date1,['PERMNO','RET']].pivot(columns='PERMNO').loc[:,'RET']

date2=date[400:]
characteristics2=dict(zip(
    date2,
    [2*data.loc[i].set_index('PERMNO').iloc[:,1:].rank()/data.loc[i].shape[0]-1 for i in date2]
))
ret2=data.loc[date2,['PERMNO','RET']].pivot(columns='PERMNO').loc[:,'RET']

In [38]:
class IPCA():
    def __init__(self,characteristics,ret):
        self.characteristics=copy(characteristics)
        self.ret=copy(ret)
        
        self.date_list=list(self.characteristics.keys())
        self.stock_list=list(self.ret.columns)
        self.Nt=dict(zip(
            self.date_list,
            [self.characteristics[i].shape[0] for i in self.date_list]
        ))
        self.chanames=list(self.characteristics[self.date_list[0]].columns)
        self.l=len(self.chanames)
        
        self.xt=pd.DataFrame(np.nan,index=self.date_list,columns=self.chanames)
        for t in self.date_list:
            self.xt.loc[t]=self.characteristics[t].T.dot(self.ret.loc[t,self.characteristics[t].index])

        self.zt_square=dict(zip(
            self.date_list,
            [self.characteristics[i].T.dot(self.characteristics[i]) for i in self.date_list]
        ))
        
        gc.collect()
    
    def fit(self,k,maxT=1000,tol=1e-3):
        self.k=k
        eigvalue,eigvector=np.linalg.eig(self.xt.T.dot(self.xt))
        self.gamma=eigvector[:,:self.k]
        
        def calc_ft(zt_square,xt,gamma):
            gamma_zts=gamma.T.dot(zt_square).dot(gamma)
            if np.linalg.det(gamma_zts)>0:
                ft=np.linalg.inv(gamma_zts).dot(gamma.T).dot(xt)
                return ft
            else:
                pass

        def calc_denomt(ft,zt_square):
            return np.kron(ft.dot(ft.T),zt_square)

        def calc_numert(ft,xt):
            return np.kron(ft,xt)

        def calc_gamma(ft):
            denom=np.zeros((self.k*self.l,self.k*self.l))
            numer=np.zeros((self.k*self.l,1))
            for t in self.date_list:
                ft_=ft.loc[t].values.reshape(self.k,1)
                if ft.loc[t].sum()!=0:
                    denom=denom+calc_denomt(ft_,self.zt_square[t])
                    numer=numer+calc_numert(ft_,self.xt.loc[t].values.reshape(self.l,1))
            if np.linalg.det(denom)>0:
                gamma=np.linalg.inv(denom).dot(numer)
            else:
                gamma=np.linalg.pinv(denom).dot(numer)
            gamma=gamma.reshape(self.k,self.l).T
            return gamma
        
        for i in range(maxT):
            self.ft=pd.DataFrame(np.nan,index=self.date_list,columns=['PC'+str(i) for i in range(1,self.k+1)])
            for t in self.date_list:
                self.ft.loc[t]=calc_ft(self.zt_square[t],self.xt.loc[t],self.gamma)

            gamma_=copy(self.gamma)
            self.gamma=calc_gamma(self.ft)

            error=((self.gamma-gamma_)**2).sum()
            if error<=tol:
                break
            else:
                print('round {}, error: {}'.format(i+1,error))

        Chol=np.linalg.cholesky(self.gamma.T.dot(self.gamma)).T
        fcov=self.ft.dropna().T.dot(self.ft.dropna())/self.ft.shape[0]
        eigvalue,Orth=np.linalg.eig(Chol.dot(fcov).dot(Chol.T))
        self.gamma=self.gamma.dot(np.linalg.inv(Chol)).dot(Orth)
        self.ft=self.ft.dot(Chol.T).dot(Orth)
        self.ft.columns=['PC'+str(i) for i in range(1,self.k+1)]
        gc.collect()
        
    def predict(self,zt):
        mu=self.ft.mean(skipna=True).values.reshape(self.k,1)
        return zt.dot(self.gamma).dot(mu)[0]

In [35]:
mod=IPCA(characteristics1,ret1)
mod.fit(6,500)

round 1, error: 15.062591011964711
round 2, error: 1.1200394525652393
round 3, error: 0.37082819664252864
round 4, error: 0.32020292540086953
round 5, error: 2.111619614971916
round 6, error: 0.939196335639493
round 7, error: 0.08965008989954124
round 8, error: 0.05636112230600393
round 9, error: 0.8897539296735502
round 10, error: 0.4437635764305531
round 11, error: 0.1109981716010662
round 12, error: 0.2507724082892738
round 13, error: 0.16485403805699875
round 14, error: 0.5667711780132234
round 15, error: 0.396070111276835
round 16, error: 0.11970495654758273
round 17, error: 0.05962824833237119
round 18, error: 0.679749386597074
round 19, error: 0.17399142562969486
round 20, error: 0.9903240023328095
round 21, error: 0.20682516591251204
round 22, error: 0.3493263922297799
round 23, error: 4.178804781418979
round 24, error: 1.1629499262883187
round 25, error: 1.3789457402503795
round 26, error: 0.6340098258602218
round 27, error: 0.11425274877329687
round 28, error: 0.0831507663191

round 222, error: 0.1926918664851942
round 223, error: 0.1641812148871783
round 224, error: 0.22227504497748152
round 225, error: 0.5291310856078799
round 226, error: 0.3106902283905342
round 227, error: 0.06372434014462311
round 228, error: 0.5053524223020378
round 229, error: 0.5618173283228962
round 230, error: 1.0683262520990153
round 231, error: 0.4927844537587364
round 232, error: 0.1569726256848004
round 233, error: 0.10073338088326561
round 234, error: 0.05111174860031044
round 235, error: 0.17538207350777069
round 236, error: 0.27482367120397533
round 237, error: 0.11673446008396762
round 238, error: 0.04639194050914887
round 239, error: 0.8480478103651408
round 240, error: 0.28247552000410897
round 241, error: 3.166474942888205
round 242, error: 1.443948600646388
round 243, error: 0.38355582866700944
round 244, error: 0.051935269768826384
round 245, error: 0.017384403364890912
round 246, error: 0.02787729936152332
round 247, error: 0.17842473001489573
round 248, error: 0.0780

round 440, error: 0.48638932604336477
round 441, error: 0.6652443552673281
round 442, error: 0.11179614953474162
round 443, error: 0.39270252089335433
round 444, error: 0.09479747841171501
round 445, error: 0.12252448396861493
round 446, error: 0.03606752700256127
round 447, error: 0.44946034347014785
round 448, error: 1.5796352134240694
round 449, error: 0.3896867336463909
round 450, error: 0.06752386466020463
round 451, error: 0.04659060039264911
round 452, error: 0.12793911931533145
round 453, error: 0.18295797091826982
round 454, error: 0.35855653854696384
round 455, error: 0.31783561984617886
round 456, error: 0.3536625558313454
round 457, error: 0.07938440656262322
round 458, error: 0.0749871420032095
round 459, error: 0.3374075016199656
round 460, error: 0.10430980554940351
round 461, error: 0.4465124759588853
round 462, error: 0.23083939361916667
round 463, error: 0.2522962750855616
round 464, error: 0.27915487804487316
round 465, error: 0.1066929708949016
round 466, error: 0.1

In [36]:
pre=copy(ret2)*0
for t in date2:
    pre.loc[t,characteristics2[t].index]=mod.predict(characteristics2[t])

In [37]:
1-((ret2-pre)**2).sum().sum()/(ret2**2).sum().sum()

0.0016261334752777357

In [21]:
class FMLARS():
    def __init__(self,characteristics,ret):
        self.characteristics=copy(characteristics)
        self.ret=copy(ret)
        self.T=self.ret.shape[0]
        
        self.date_list=list(self.characteristics.keys())
        self.stock_list=list(self.ret.columns)
        self.Nt=dict(zip(
            self.date_list,
            [self.characteristics[t].shape[0] for t in self.date_list]
        ))
        self.chanames=list(self.characteristics[self.date_list[0]].columns)
        self.l=len(self.chanames)
        
        self.xt=dict(zip(
            self.date_list,
            [
                self.characteristics[t].T.dot(self.ret.loc[t,self.characteristics[t].index]).values.reshape(self.l,1)/self.Nt[t]
                for t in self.date_list
            ]
        ))
        
        self.zt_square=dict(zip(
            self.date_list,
            [self.characteristics[i].T.dot(self.characteristics[i]).values for i in self.date_list]
        ))
        
        gc.collect()
        
    
    def fit(self,phi2,lamb):
        
        def Icrp(Ak,t,lamb):
            icrp=np.zeros((self.l,self.l))
            k=len(Ak)
            try:
                icrp[np.ix_(Ak,Ak)]=np.linalg.inv(self.zt_square[t][np.ix_(Ak,Ak)]+lamb*self.Nt[t]*np.eye(k,k))
            except:
                icrp[np.ix_(Ak,Ak)]=np.linalg.pinv(self.zt_square[t][np.ix_(Ak,Ak)]+lamb*self.Nt[t]*np.eye(k,k))
            return icrp

        def calc_gamma_t(Ak,proding_list,t,lamb):
            gamma_t=self.Nt[t]*Icrp(Ak,t,lamb).dot(proding_list[t]).dot(self.xt[t])
            return gamma_t

        def solve_alpha(coef2,coef1,cons):
            if coef2==0:
                return np.nan
            else:
                x1=(-2*coef1+np.sqrt(4*coef1**2-4*coef2*cons))/(2*coef2)
                x2=(-2*coef1-np.sqrt(4*coef1**2-4*coef2*cons))/(2*coef2)
                if 0<x1<1:
                    return(float(x1))
                else:
                    return(float(x2))

        def calc_NW_tvalue(factor_value):
            tsmod=sm.OLS(endog=factor_value, exog=np.ones(self.T), hasconst=False)\
                .fit(cov_type='HAC',cov_kwds={'maxlags':6})
            return tsmod.tvalues[0]

        self.lamb=lamb
        #k=0
        self.Ak_list=[]
        self.alpha_list=[]
        self.Ak_list.append(pd.DataFrame(
                    sum([(self.Nt[t]*self.xt[t])**2 for t in self.date_list])\
                    +phi2*sum([self.Nt[t]*self.xt[t] for t in self.date_list])**2
                ).idxmax().iloc[0])

        proding_list=dict(zip(
            self.date_list,
            [np.eye(self.l) for t in self.date_list]
        ))
        
        #k>0
        print('Estimating FM-LARS')
        for k in trange(self.l):
            try:
                Zr=[]
                Zsq_gamma=[]
                Ak=self.Ak_list[0:k+1]
                Ak_1=self.Ak_list[0:k]
                for t in self.date_list:
                    if k>0:
                        proding_list[t]=proding_list[t]\
                        .dot(np.eye(self.l)-self.alpha_list[k-1]*self.zt_square[t].dot(Icrp(Ak_1,t,self.lamb)))
                gamma_t_list=dict(zip(
                    self.date_list,
                    [calc_gamma_t(Ak,proding_list,t,self.lamb) for t in self.date_list]
                ))
                
                for t in self.date_list:
                    Zr.append(self.Nt[t]*proding_list[t].dot(self.xt[t]))
                    Zsq_gamma.append(self.zt_square[t].dot(gamma_t_list[t]))

                coef2=sum([Zg**2 for Zg in Zsq_gamma])\
                        +phi2*sum(Zsq_gamma)**2
                coef2=coef2-coef2[self.Ak_list[0]]
                coef1=-sum([zr * Zg for (zr,Zg) in zip(Zr,Zsq_gamma)])\
                        -phi2*sum(Zr)*sum(Zsq_gamma)                
                coef1=coef1-coef1[self.Ak_list[0]]
                cons=sum([zr**2 for zr in Zr])+phi2*sum(Zr)**2                
                cons=cons-cons[self.Ak_list[0]]
                
                
                solving_alpha_list=np.array([solve_alpha(c2,c1,c) for (c2,c1,c) in zip(coef2,coef1,cons)])
                solving_alpha_list[Ak]=np.nan

                if k<self.l-1:
                    j_=pd.DataFrame(solving_alpha_list).idxmin().iloc[0]
                    alpha_k=solving_alpha_list[j_]
                else:
                    j_=Ak[len(Ak)-1]
                    alpha_k=1
                self.Ak_list.append(j_)
                self.alpha_list.append(alpha_k)
                
                factor_k=pd.DataFrame(np.nan,index=self.date_list,columns=self.chanames)
                for t in self.date_list:
                    factor_k.loc[t]=gamma_t_list[t].reshape(self.l,)
                if k==0:
                    self.factor_list=[self.alpha_list[k]*factor_k]
                else:
                    self.factor_list.append(self.factor_list[k-1]+self.alpha_list[k]*factor_k)
            except:
                break
        
        self.maxk=len(self.factor_list)
        
        def get_mu_hat(k):
            mu_hat=pd.DataFrame(zip(\
                    [np.mean(self.factor_list[k].iloc[:,i]) for i in range(self.l)],\
                    [calc_NW_tvalue(self.factor_list[k].iloc[:,i]) for i in range(self.l)]
                        ),columns=['risk premium','t value'],\
                       index=self.chanames).fillna(0)
            mu_hat=mu_hat[mu_hat.iloc[:,1]!=0]
            return(mu_hat)
        
        self.mu_hat=[get_mu_hat(k) for k in range(self.maxk)]
                
    def predict(self,Z_t,maxk=None):
        if maxk==None:
            maxk=self.maxk
        prediction=[]
        for k in range(maxk):
            r_hat=Z_t.loc[:,self.mu_hat[k].index].dot(self.mu_hat[k].iloc[:,0].values)
            prediction.append(r_hat)
        return prediction

In [7]:
mod=FMLARS(characteristics1,ret1)

In [15]:
mod.fit(5,0)

  6%|████▌                                                                              | 2/36 [00:00<00:02, 13.35it/s]

Estimating FM-LARS


<ipython-input-5-210090a915e7>:51: RuntimeWarning: invalid value encountered in sqrt
  x1=(-2*coef1+np.sqrt(4*coef1**2-4*coef2*cons))/(2*coef2)
<ipython-input-5-210090a915e7>:52: RuntimeWarning: invalid value encountered in sqrt
  x2=(-2*coef1-np.sqrt(4*coef1**2-4*coef2*cons))/(2*coef2)
100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [00:08<00:00,  4.34it/s]


In [16]:
prediction=[]
for i in range(mod.maxk):
    prediction.append(copy(ret2)*0)
    
for t in trange(len(date2)):
    key=date2[t]
    pre=mod.predict(characteristics2[key])
    for i in range(mod.maxk):
        prediction[i].loc[key,characteristics2[key].index]=pre[i]

100%|████████████████████████████████████████████████████████████████████████████████| 223/223 [00:18<00:00, 11.75it/s]


In [20]:
k=30
100*(1-((ret2-prediction[k])**2).sum().sum()/(ret2**2).sum().sum())

0.2666052079092873